#                                            Design Safe Use Cases- Using USGS API to map earthquakes on world map

 Welcome! This Jupyter notebook will walk through how to access a specific API (Application Programming interaface) that is available through USGS that details earthquakes over a certain magntiude that occured over a specific time period. The goal of this notebook is to take the USGS hourly/weekly/monthly earthquake RSS feed ( https://earthquake.usgs.gov/earthquakes/feed/) and plot the earthquakes and their relevant magnitudes using a Python Package called Folium (https://python-visualization.github.io/folium/).
 
 ## Requirements
folium  
geopandas  
requests  
json  
numpy

## Install packages

To begin building, we will install first install the Folium Python Package which we will use later on. To install them, we will use the "pip install" framework. The other required packages are already part of Python, so there is no reason to separately install them. 

In [7]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


## Import packages

Next to set-up the notebook we will call various packages and modules using the " import function". This will allow us to more seamlessly use the packages as needed throughout the notebook. Packages inclue NumPy (https://numpy.org/) and Pandas (https://pandas.pydata.org/

In [8]:
import requests
import numpy
import json
import pandas as pd
import folium


## Make an API Call 

We then call the USGS API via the the following URL ( details can be found here https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php). Calling the API directly can be helpful as the URL is constantly updated and will provide the most up to date information. This example references the earthquake feed for events that occured in the last month that are over Mw= 2.5. We will convert the url into a data parseable form, JSON (JavaScript Object Notation). 

In [9]:
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson'
r = requests.get(url)
json_data= r.json()
json_data


{'type': 'FeatureCollection',
 'metadata': {'generated': 1684109047000,
  'url': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson',
  'title': 'USGS Magnitude 2.5+ Earthquakes, Past Month',
  'status': 200,
  'api': '1.10.3',
  'count': 1725},
 'features': [{'type': 'Feature',
   'properties': {'mag': 2.59,
    'place': '5 km SW of Guánica, Puerto Rico',
    'time': 1684106620060,
    'updated': 1684108357370,
    'tz': None,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/pr71409423',
    'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/pr71409423.geojson',
    'felt': None,
    'cdi': None,
    'mmi': None,
    'alert': None,
    'status': 'reviewed',
    'tsunami': 0,
    'sig': 103,
    'net': 'pr',
    'code': '71409423',
    'ids': ',pr71409423,',
    'sources': ',pr,',
    'types': ',origin,phase-data,',
    'nst': 10,
    'dmin': 0.04528,
    'rms': 0.08,
    'gap': 212,
    'magType': 'md',
    'type': 'earthquake',


## Understanding JSON keys

Once we have the data in the JSON format, we need to identify what data we need from the output to map the data. JSON "keys" are found on the left side of the colons, such as "geometry" and "features". Values for the keys are found on the right side of the colons and give details about those keys. To build a map of the earthquakes, we will pull the coordinate information from the "geometry" key, as well as the title of the earthquake from the "properties key". The values are collected through a "for loop" and are saved for future use. 

In [10]:
lat1 = []
lon1 = []
captions = []
for earthquake in json_data['features']:
    lat,lon,depth= earthquake['geometry']['coordinates']
    longitude=(lon)
    latitude = (lat)
    lat1.append(lat)
    lon1.append(lon)
    labelmarkers= earthquake['properties']['title']
    names=(labelmarkers)
    captions.append(names)


## List of sites and magnitudes 

Using a simple zip function, we are able to put the coordinates and the title/magnitude for the earthquakes into a list.



In [11]:
mapinfo_list  = list (zip(lat1,lon1, captions))
mapinfo_list

[(-66.9471666666667, 17.9455, 'M 2.6 - 5 km SW of Guánica, Puerto Rico'),
 (-151.0346, 61.154, 'M 2.6 - 2 km ENE of Beluga, Alaska'),
 (148.487, 45.416, 'M 4.9 - 52 km ENE of Kuril’sk, Russia'),
 (-178.2536, -17.9791, 'M 4.5 - 257 km E of Levuka, Fiji'),
 (-121.1616669, 36.5241661, 'M 2.9 - 2km WSW of Pinnacles, CA'),
 (-121.1051636, 40.1988335, 'M 2.8 - 4km NW of Canyondam, CA'),
 (-121.0699997, 40.208168, 'M 2.7 - 4km S of East Shore, CA'),
 (7.2892, 75.3895, 'M 4.6 - Greenland Sea'),
 (-113.5394, 36.4558, 'M 2.6 - 56 km SE of Scenic, Arizona'),
 (-73.0975, 6.8608, 'M 4.6 - 14 km N of Jordán, Colombia'),
 (139.1763, 33.3737, 'M 5.1 - 146 km S of Shimoda, Japan'),
 (-104.1564, 31.5515, 'M 2.8 - 43 km NW of Toyah, Texas'),
 (126.925, 0.4855, 'M 5.1 - Molucca Sea'),
 (-155.461669921875, 19.1578330993652, 'M 2.5 - 5 km SSE of Pāhala, Hawaii'),
 (-155.479827880859, 19.1591663360596, 'M 2.5 - 4 km S of Pāhala, Hawaii'),
 (139.2095, 33.2302, 'M 4.5 - 162 km S of Shimoda, Japan'),
 (-116.441

## Dataframe creation 

From the list we are able to convert the information to a dateframe which makes data easier to digest. 



In [12]:
df = pd.DataFrame(mapinfo_list,columns =['latitude','longitude','title'])
df

,latitude,longitude,title
0,-66.947167,17.945500,"M 2.6 - 5 km SW of Guánica, Puerto Rico"
1,-151.034600,61.154000,"M 2.6 - 2 km ENE of Beluga, Alaska"
2,148.487000,45.416000,"M 4.9 - 52 km ENE of Kuril’sk, Russia"
3,-178.253600,-17.979100,"M 4.5 - 257 km E of Levuka, Fiji"
4,-121.161667,36.524166,"M 2.9 - 2km WSW of Pinnacles, CA"
...,...,...,...
1720,-66.956667,17.943500,M 2.6 - Puerto Rico region
1721,-67.387667,18.404833,"M 3.4 - Mona Passage, Puerto Rico"
1722,-66.846000,17.925167,"M 3.0 - 7 km SE of Maria Antonia, Puerto Rico"
1723,-175.109100,51.760800,"M 2.6 - 79 km SW of Atka, Alaska"


## Map Creation 

The final step is to create the map with the earthquake information collated so far. Using Folium documentation (https://python-visualization.github.io/folium/) we will set up a simple map, using locations from the dataframe. 

In [13]:
for index, location_info in df.iterrows():
    my_map=folium.Map(locations=[location_info["longitude"],location_info["latitude"]],zoom_start=10, control_scale=True,tiles= 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                        attr='ESRI')
my_map

 The final folium cell allows the user to add the marker/title and other changes to the base map previously created. Here we added markers and titles to the map. The final map shows the earthquake locations and pop-ups with the name and magintude. Given your project, you could extract various data from the json and create relevant maps/figures with that data.

In [14]:
for index, location_info in df.iterrows():
    folium.Marker([location_info["longitude"], location_info["latitude"]],popup=location_info["title"], color ='purple').add_to(my_map)
my_map